In [24]:
import math
import itertools

In [25]:
states = ['G', 'B']
observations_str = "L, M, M, H, M, L, M, M, L, L, H, M, M, H, L"
observations = [obs.strip() for obs in observations_str.split(',')]
N = len(states)
T = len(observations)
state_to_idx = {'G': 0, 'B': 1}
obs_to_idx = {'L': 0, 'M': 1, 'H': 2}
idx_to_state = {0: 'G', 1: 'B'}
obs_indices = [obs_to_idx[obs] for obs in observations]

In [26]:
B = [[0.6, 0.3, 0.1],
     [0.2, 0.3, 0.5]]

log_B = [[math.log(p) for p in row] for row in B]

In [27]:
def run_viterbi(pi_param, A_param, B_param_log, obs_indices_param):
    N_local = len(pi_param)
    T_local = len(obs_indices_param)

    if any(p == 0 for p in pi_param) or any(p == 0 for row in A_param for p in row):
         return None

    try:
        log_pi_local = [math.log(p) for p in pi_param]
        log_A_local = [[math.log(p) for p in row] for row in A_param]
    except ValueError:
        return None

    viterbi = [[0.0] * T_local for _ in range(N_local)]
    backpointer = [[0] * T_local for _ in range(N_local)]

    first_obs_idx_local = obs_indices_param[0]
    for s in range(N_local):
         init_val = log_pi_local[s] + B_param_log[s][first_obs_idx_local]
         viterbi[s][0] = init_val if init_val > -float('inf') else -float('inf')
         backpointer[s][0] = 0

    for t in range(1, T_local):
        current_obs_idx_local = obs_indices_param[t]
        for j in range(N_local):
            max_log_prob = -float('inf')
            best_prev_state = -1
            for i in range(N_local):
                if viterbi[i][t-1] > -float('inf'):
                     prob = viterbi[i][t-1] + log_A_local[i][j]
                     if prob > max_log_prob:
                         max_log_prob = prob
                         best_prev_state = i

            if max_log_prob > -float('inf'):
                final_log_prob = max_log_prob + B_param_log[j][current_obs_idx_local]
                viterbi[j][t] = final_log_prob if final_log_prob > -float('inf') else -float('inf')
            else:
                viterbi[j][t] = -float('inf')

            backpointer[j][t] = best_prev_state

    best_last_state_idx = -1
    max_final_log_prob = -float('inf')
    for s in range(N_local):
        final_prob = viterbi[s][T_local-1]
        if final_prob > max_final_log_prob:
            max_final_log_prob = final_prob
            best_last_state_idx = s

    if best_last_state_idx == -1:
        return None

    best_path_indices = [0] * T_local
    best_path_indices[T_local-1] = best_last_state_idx
    for t in range(T_local-2, -1, -1):
        current_state_idx_in_path = best_path_indices[t+1]
        prev_state_idx_in_path = backpointer[current_state_idx_in_path][t+1]
        if prev_state_idx_in_path == -1 and t != T_local-2 :
             return None
        best_path_indices[t] = prev_state_idx_in_path if prev_state_idx_in_path != -1 else 0


    best_path_states_local = [idx_to_state[i] for i in best_path_indices]
    return "".join(best_path_states_local)

In [28]:
pi_options = [[0.5, 0.5], [0.9, 0.1], [0.1, 0.9], [0.7, 0.3], [0.3, 0.7]]

A_diagonals = [
    [0.9, 0.9], [0.8, 0.8], [0.7, 0.7], [0.6, 0.6], [0.5, 0.5], [0.4, 0.4],
    [0.8, 0.9], [0.7, 0.9], [0.6, 0.9], [0.5, 0.9],
    [0.9, 0.8], [0.9, 0.7], [0.9, 0.6], [0.9, 0.5],
    [0.9, 0.4], [0.4, 0.9], [0.8, 0.5], [0.5, 0.8]
]

unique_results = set()

incorrect_sequences = {
    "GGGBGGGGGGBBBBG", "GBBBGGGGGGBBBBG", "GBBBBBBBBBBBBBB",
    "GGGGGGGGGGBBBBG", "GGGGGGGGGGBBBBB", "GGGBBGGGGGBBBBB",
    "GGGGGGGGGBBBBBB", "GGGGGGGGBBBBBBB", "BBBBBBBBBBBBBBB",
    "BBBBBGGGGGBBBBB", "BBBBBGGGGGBBBBG", "BBBBGGGGGGBBBBG",
    "BBBBGGGGGGGGGGG", "BBGBGGBGGGBGGBG", "BGGBGGGGGGBGGBG",
    "BGGGGGGGGGGGGGG", "GBBBBGGGGGBBBBG", "GBGBGGBGGGBGGBG",
    "GGGGGGGGGGGGGGG"
}

In [29]:
print("--- Запуск Расширенного Перебора ---")
count = 0
for pi_val in pi_options:
    for diag in A_diagonals:
        p_gg, p_bb = diag[0], diag[1]
        p_gg = max(0.001, min(0.999, p_gg))
        p_bb = max(0.001, min(0.999, p_bb))
        p_gb = 1.0 - p_gg
        p_bg = 1.0 - p_bb
        A_val = [[p_gg, p_gb], [p_bg, p_bb]]

        pi_val_rounded = [round(p, 10) for p in pi_val]
        A_val_rounded = [[round(p, 10) for p in row] for row in A_val]

        result = run_viterbi(pi_val_rounded, A_val_rounded, log_B, obs_indices)
        if result:
            unique_results.add(result)
        count += 1

--- Запуск Расширенного Перебора ---


In [30]:
print(f"\n--- Перебор Завершен ({count} комбинаций) ---")
print(f"Найдено уникальных последовательностей: {len(unique_results)}")


--- Перебор Завершен (90 комбинаций) ---
Найдено уникальных последовательностей: 17


In [31]:

print("\n--- Новые Уникальные Последовательности (Кандидаты) ---")
new_candidates_found = False
for res in sorted(list(unique_results)):
    if res not in incorrect_sequences:
        print(res)
        new_candidates_found = True

if not new_candidates_found:
    print("Новых уникальных последовательностей, отличных от неверных, не найдено.")


--- Новые Уникальные Последовательности (Кандидаты) ---
GGGBGGGGGGBGGBG
